In [ ]:
from kbmod.work_unit import WorkUnit
from astropy.table import Table
from kbmod.run_search import SearchRunner
import numpy as np

In [ ]:
t = Table.read("inframe_fakes_b1h.ecsv")

## Setup

before we run a targeted search for a specific object, we need to pick an object to test on!

In [ ]:
orbids = np.unique(t["ORBITID"])
orbids

These are our unique orbit ids. To select the rows with all the possible, let's select a specific orbit id as `oid` and get all the positions associated with that object.

In [ ]:
oid = orbids[2]
t0 = t[t["ORBITID"] == oid]
t0

Now we can load in our `WorkUnit` for the b1h field and convert the longitude and latitude coordinates into pixel coordinates.

In [ ]:
wu = WorkUnit.from_fits("/epyc/projects/kbmod/data/20210908_B1h_047_test_data/20210908_B1h_047.wu")

In [ ]:
xs, ys = wu.get_pixel_coordinates(t0["RA"], t0["DEC"])

Now we have a list of x and y coordinates, `xs` and `ys`. Before we go any further, we should confirm that given orbit is going to be roughly linear in pixel trajectory terms. Basically we want to make sure that the observations are lined up in such a way that `KBMOD` will actually find the object while searching the image. to do this, we need to calculate the changes in pixels in the x and y direction and compare the mean, maximum and minimum.

In [ ]:
dx = []
for i in range(len(xs) - 2):
    dx.append(xs[i+1] - xs[i])

dy = []
for i in range(len(ys) - 2):
    dy.append(ys[i+1] - ys[i])

Generally, the min and max for the `dx` and `dy` shouldn't be much greater than +-1. If it is more than that, this orbit id might not be a good choice for actually searching for in `KBMOD`.

In [ ]:
print(np.mean(dx), np.min(dx), np.max(dx))
print(np.min(dx) - np.mean(dx), np.max(dx) - np.mean(dx))

In [ ]:
print(np.mean(dy), np.min(dy), np.max(dy))
print(np.min(dy) - np.mean(dy), np.max(dy) - np.mean(dy))

now we have to calculate the pixels per day velocity for both the x and y. To do this we'll take the difference between the first and last x and y positions and divide the total time elapsed between the start and end of the search.

In [ ]:
time_diff = t0["mjd_mid"][-1] - t0["mjd_mid"][0]

In [ ]:
mean_vx = (xs[-1] - xs[0]) / time_diff
mean_vx

In [ ]:
mean_vy = (ys[-1] - ys[0]) / time_diff
mean_vy

Now we can use the `mean_vx` and `mean_vy` as a starting point to try and guess where the velocity is and where `KBMOD` will find the object.

I put in a simple search that worked to find `orbids[0]` here but feel free to play around with this generator config. The step values is the difference in velocities that `KBMOD` will search around.

In [ ]:
gen = {
    "name": "VelocityGridSearch",
    "vx_steps": 25,
    "min_vx": mean_vx - .125,
    "max_vx": mean_vx + .125,
    "vy_steps": 25,
    "min_vy": mean_vy - .125,
    "max_vy": mean_vy + .125,
}

In [ ]:
wu.config.set("generator_config", gen)
wu.config.set("do_clustering", False)
wu.config.set("sigmaG_filter", False)
# wu.config.set("peak_offset_max", 6)
# wu.config.set("predictive_line_cluster", True)
wu.config.set("lh_level", 10.0)

In [ ]:
wu.config.set("result_filename", f"./{oid}_pencil.ecsv")

In [ ]:
wu.reprojection_frame = "original"

In [ ]:
r = SearchRunner().run_search_from_work_unit(wu)

if you need to quit out of the kernel and reload with the saved result file feel free to comment this out.

In [ ]:
# from kbmod.results import Results
# r = Results.read_table(f"{oid}_pencil.ecsv")

In [ ]:
r

If the following check doesn't result in any rows than it's likely that the search didn't find the object in question. You can still check around in the above table to see if there's something close to the original object in starting x,y and vx,vy but this is probably the fastest way to check if the object was recovered!

In [ ]:
r[(r["x"] >= xs[0] - 1) & (r["x"] <= xs[0] + 1) & (r["y"] >= ys[0] - 1) & (r["y"] <= ys[0] + 1)]